# Create data for stacking code testing

In [1]:
import numpy as np
from pathlib import Path
from astropy.io import fits
from astropy.table import Table, vstack, hstack, unique,join
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
import pandas as pd
import sys

# sys.path.append("/global/u2/b/bid13/VI/prospect/py")

import desispec.coaddition
import desispec.io
import desispec.spectra
# from desitarget.cmx.cmx_targetmask import cmx_mask
# from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask
from desitarget.sv3.sv3_targetmask import desi_mask, bgs_mask
from prospect import utilities,plotframes, myspecselect, myspecupdate
from desispec.coaddition import coadd_cameras
from prospect import utilities ,plotframes

sys.path.append("../")
from importlib import reload
import stack_spectra
reload(stack_spectra)
# from prospect_my import plotframes

<module 'stack_spectra' from '../stack_spectra.py'>

In [2]:
data_path = Path("/global/cfs/cdirs/desi/spectro/redux/denali/tiles/cumulative")
my_path = Path("/global/cscratch1/sd/bid13/stack_spectra")

In [3]:
tiles = ["80606","80608", "80610","80621", "80623", "80669", "80673", "80675",
         "80677", "80681", "80683", "80685", "80687", "80689", "80691", "80693",
         "80695", "80697", "80699", "80701", "80703", "80705", "80707", "80709", "80711"]


In [4]:
z_table = Table()
spectra = desispec.spectra.Spectra()
spec_flux = []
spec_wave = []
spec_ivar = []
sky_ivar = []

for tile in tiles:
    tile_path = data_path / tile
    for file_path in tile_path.glob("*/zbest-*"):

        z_petal = Table.read(file_path, hdu="ZBEST")
        fibermap_petal = Table.read(file_path, hdu="FIBERMAP")[0:500]

        merged_table = join(z_petal, fibermap_petal, keys="TARGETID", metadata_conflicts="silent")
        if "SV1_DESI_TARGET" in merged_table.keys():
            merged_table["DESI_TARGET"] = merged_table["SV1_DESI_TARGET"]
        elif "SV2_DESI_TARGET" in merged_table.keys():
            merged_table["DESI_TARGET"] = merged_table["SV2_DESI_TARGET"]
        else:
            merged_table["DESI_TARGET"] = merged_table["SV3_DESI_TARGET"]
            
        mask = ((merged_table["DESI_TARGET"] & desi_mask.mask("ELG"))>0) & (merged_table["FIBERSTATUS"] == 0)
        mask &= (merged_table["Z"]>=1.3) & (merged_table["Z"]<=1.4)
        mask &= (merged_table["SPECTYPE"]=="GALAXY")
        mask &= (merged_table["DELTACHI2"]>15)
        
        sky_mask = (merged_table["OBJTYPE"]=="SKY")&(merged_table["FIBERSTATUS"] == 0)
        sky_table = merged_table[sky_mask]
        
        merged_table = merged_table[mask]
        
        
        object_coord = SkyCoord(ra=merged_table["TARGET_RA"]*u.degree, dec=merged_table["TARGET_DEC"]*u.degree)
        sky_coord = SkyCoord(ra=sky_table["TARGET_RA"]*u.degree, dec=sky_table["TARGET_DEC"]*u.degree)
        idx, d2d, d3d = object_coord.match_to_catalog_sky(sky_coord)
        
        spec = desispec.io.read_spectra(str(file_path).replace("zbest","coadd") )
        flux, wave, ivar = stack_spectra._coadd_cameras(spec.flux, spec.wave, spec.ivar)
        z_table = vstack([z_table, merged_table])
        spec_flux.append(flux[mask])
        spec_ivar.append(ivar[mask])
        spec_wave = wave
        sky_ivar.append(ivar[idx])

INFO:spectra.py:253:read_spectra: iotime 0.493 sec to read coadd-6-80606-thru20201219.fits at 2021-04-25T18:27:13.856640
INFO:spectra.py:253:read_spectra: iotime 0.590 sec to read coadd-8-80606-thru20201219.fits at 2021-04-25T18:27:18.868624
INFO:spectra.py:253:read_spectra: iotime 0.468 sec to read coadd-2-80606-thru20201219.fits at 2021-04-25T18:27:23.828690
INFO:spectra.py:253:read_spectra: iotime 0.468 sec to read coadd-5-80606-thru20201219.fits at 2021-04-25T18:27:28.642838
INFO:spectra.py:253:read_spectra: iotime 0.482 sec to read coadd-3-80606-thru20201219.fits at 2021-04-25T18:27:33.333579
INFO:spectra.py:253:read_spectra: iotime 0.476 sec to read coadd-1-80606-thru20201219.fits at 2021-04-25T18:27:38.069302
INFO:spectra.py:253:read_spectra: iotime 0.469 sec to read coadd-9-80606-thru20201219.fits at 2021-04-25T18:27:43.109996
INFO:spectra.py:253:read_spectra: iotime 0.423 sec to read coadd-4-80606-thru20201219.fits at 2021-04-25T18:27:47.764818
INFO:spectra.py:253:read_spectra

In [5]:
np.savez(my_path / "ELG_1.3_1.4_v0", flux = np.concatenate(spec_flux),
         ivar = np.concatenate(spec_ivar), wave= np.array(wave),
         sky_ivar= np.concatenate(sky_ivar),
        z_table=z_table.as_array())

In [6]:
len(z_table)

3321